In [ ]:
import pandas as pd

fve_df = pd.read_csv('data/FVE_Vresovice_2024_15min_extrapolated.csv', sep=',')
fve_df['datetime'] = pd.to_datetime(fve_df['DT'], format='ISO8601')
fve_df = fve_df.set_index('datetime')

fve_df['Výroba [kW]'] = fve_df['FVE power [MW]'] * 1000

fve_df = fve_df[['Výroba [kW]']]
FVE = fve_df[fve_df.index.year == 2024]

In [ ]:
vdt2024 = pd.read_excel("data/OTE_2024.xlsx", sheet_name="VDT (EUR)", skiprows=5, usecols="A:L")
dt2024 = pd.read_excel("data/OTE_2024.xlsx", sheet_name="DT ČR", skiprows=5, usecols="A:L")

In [ ]:
vdt2024['datetime'] = pd.to_datetime(vdt2024['Den'])
vdt2024['datetime'] = vdt2024['datetime'] + pd.to_timedelta(vdt2024['Hodina'] - 1, unit='h')

vdt2024.sort_values('datetime', inplace=True)
vdt2024.drop_duplicates(subset=['datetime'], keep='first', inplace=True)

vdt2024 = vdt2024[['datetime', 'Vážený průměr cen (EUR/MWh)']]
vdt2024.set_index('datetime', inplace=True)

vdt_15min = vdt2024.resample('15T').ffill()

vdt_15min.reset_index(inplace=True)
vdt_15min['Den'] = vdt_15min['datetime'].dt.normalize()
vdt_15min['Perioda'] = vdt_15min.groupby(vdt_15min['datetime'].dt.date).cumcount() + 1

vdt2024_formatted = vdt_15min[['datetime', 'Den', 'Perioda', 'Vážený průměr cen (EUR/MWh)']]
vdt2024_formatted.fillna(method='ffill', inplace=True)

VDT = vdt2024_formatted[vdt2024_formatted['datetime'].dt.year == 2024]
VDT.set_index('datetime', inplace=True)

In [ ]:
dt2024['datetime'] = pd.to_datetime(dt2024['Den'])

dt2024['datetime'] = dt2024['datetime'] + pd.to_timedelta(dt2024['Hodina'] - 1, unit='h')

dt2024.sort_values('datetime', inplace=True)
dt2024.drop_duplicates(subset=['datetime'], keep='first', inplace=True)

dt2024_subset = dt2024[['datetime', 'Marginální cena ČR (EUR/MWh)']]
dt2024_subset.set_index('datetime', inplace=True)

dt_15min = dt2024_subset.resample('15min').ffill()

dt_15min.reset_index(inplace=True)

dt_15min['Den'] = dt_15min['datetime'].dt.normalize()
dt_15min['Perioda'] = dt_15min.groupby(dt_15min['datetime'].dt.date).cumcount() + 1

dt2024_formatted = dt_15min[['datetime', 'Den', 'Perioda', 'Marginální cena ČR (EUR/MWh)']]

last_known_price = 83.2 

new_rows_data = [
    {
        'datetime': pd.to_datetime('2024-12-31 23:15:00'), 
        'Den': pd.to_datetime('2024-12-31'), 
        'Perioda': 94, 
        'Marginální cena ČR (EUR/MWh)': last_known_price
    },
    {
        'datetime': pd.to_datetime('2024-12-31 23:30:00'), 
        'Den': pd.to_datetime('2024-12-31'), 
        'Perioda': 95, 
        'Marginální cena ČR (EUR/MWh)': last_known_price
    },
    {
        'datetime': pd.to_datetime('2024-12-31 23:45:00'), 
        'Den': pd.to_datetime('2024-12-31'), 
        'Perioda': 96, 
        'Marginální cena ČR (EUR/MWh)': last_known_price
    }
]

new_rows_df = pd.DataFrame(new_rows_data)

DT = pd.concat([dt2024_formatted, new_rows_df], ignore_index=True)
DT.set_index('datetime', inplace=True)

In [1]:
import pandas as pd

VDT = (pd.read_excel('data/data.xlsx', sheet_name='VDT (R2W)')).set_index('datetime', inplace=False)
DT = (pd.read_excel('data/data.xlsx', sheet_name='DT (R2W)')).set_index('datetime', inplace=False)
FVE = (pd.read_excel('data/data.xlsx', sheet_name='FVE (R2W)')).set_index('datetime', inplace=False)
DT = DT[DT.index <= '2025-06-30']
VDT = VDT[VDT.index <= '2025-06-30']
FVE = FVE[FVE.index <= '2025-06-30']
VDT['Vážený průměr cen (EUR/MWh)'].fillna(method='ffill', inplace=True)

C:\Users\Tamerlan.Ismakatov\AppData\Local\Temp\ipykernel_34576\1703595386.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  VDT['Vážený průměr cen (EUR/MWh)'].fillna(method='ffill', inplace=True)
C:\Users\Tamerlan.Ismakatov\AppData\Local\Temp\ipykernel_34576\1703595386.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  VDT['Vážený průměr cen (EUR/MWh)'].fillna(method='ffill', inplace=True)


In [2]:
from FveBessOptimizer import analyze_projects, FVE_BESS_Optimizer, export_full_report_to_excel

PROJECTS_CONFIG = {
    'phase2': {
        'name': 'phase2',
        'fve_power_mw': 4.124,           
        'fve_scale_factor': 4.124 / 1.6,  
        'bess_power_mw': 3.2,
        'bess_capacity_mwh': 6.4,
        'export_limit_mw': 6.5,      
        'import_limit_mw': 3.21,      
        'efficiency': 0.85,
        'max_cycles': 2          
    },
}
results = analyze_projects(DT, VDT, FVE, projects_config=PROJECTS_CONFIG,
                             analysis_period_days=365, show_daily_reports=False)


Successfully analyzed 363 days
Annual income: 649,392 EUR


In [4]:
for i in results.keys():
    optimizer_instance = FVE_BESS_Optimizer(PROJECTS_CONFIG[i])

    project_key_to_export = i
    project_data_to_export = results[project_key_to_export]
            
    excel_filename = f"full_report_{project_key_to_export}.xlsx"
    export_full_report_to_excel(project_data_to_export, DT, VDT, optimizer_instance, excel_filename, with_plots=True)


Creating Excel report 'full_report_phase2.xlsx' for project 'phase2'...
✅ Report 'full_report_phase2.xlsx' was successfully created!
